In [1]:
from edgeworth_env import *

2024-04-24 14:54:20,534	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
env = OligopolyMarket()

In [3]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [4]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [5]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-04-24 14:54:22,490	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8266


In [6]:
agent_ids = OligopolyMarket()._agent_ids
sym_policies = {agent_id: f"policy_firm_0" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = asym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(OligopolyMarket)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 12000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )


tuner.fit()

2024-04-24 14:54:23,545	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1548851) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1548851) 2024-04-24 14:54:26,838	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 640.0, 'num_env_steps_trained': 1024.0, 'total_loss': 49.26491092046102}, 'policy_firm_1': {'total_loss': 49.26491092046102, 'policy_loss': -0.03319768222281709, 'vf_loss': 9.88283987045288, 'vf_loss_unclipped': 156997604.26666668, 'vf_explained_var': 1.6028682390848795e-06, 'entropy': 1.968670613070329, 'mean_kl_loss': 0.015235520738315245, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.30000001192092896}, 'policy_firm_4': {'total_loss': 9.847158364454906, 'policy_loss': -0.030755163628297548, 'vf_loss': 9.873051758607229, 'vf_loss_unclipped': 152483923.8, 'vf_explained_var': -2.337247133255005e-06, 'entropy': 2.10807353357474, 'mean_kl_loss': 0.010803722530666467, 'default_optimizer_lr': 5.0000000000000016e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.450000017